Notebook Adapted from: https://github.com/ShirinG/blog_posts_prep/blob/master/twitter/twitter_codecentric.Rmd

If you are missing libraries, you can install with: 

In [ ]:
install.packages("tidyverse")
install.packages("tidytext")
install.packages("twitteR")
install.packages("data.table")

Load R Libraries for Analysis

In [ ]:
library(tidyverse)
library(tidytext)
library(twitteR)
library("data.table")

Define Your Application Key

In [1]:
consumerKey = "INSERT KEY HERE"
consumerSecret = "INSERT SECRET KEY HERE"
accessToken = "INSERT TOKEN HERE"
accessSecret = "INSERT SECRET TOKEN HERE"

Establish your credentials with Twitter

In [2]:
setup_twitter_oauth(consumer_key = consumerKey, 
                    consumer_secret = consumerSecret, 
                    access_token = accessToken, 
                    access_secret = accessSecret)

NameError: name 'setup_twitter_oauth' is not defined

Define some Utility Functions

In [ ]:
# get follower names
get_follower_list <- function(userName){
  # get user data
  twitterUser <- getUser(userName)
  twitterUserFollowerIDs<-twitterUser$getFollowers(retryOnRateLimit=1)

  # extract the list of followers
  return (sapply(twitterUserFollowerIDs,screenName))
}



# get follower names
get_friend_list <- function(userName){
  # get user data
  twitterUser <- getUser(userName)
  twitterUserFriendIDs<-twitterUser$getFriends(retryOnRateLimit=1)

  # extract the list of followers
  return (sapply(twitterUserFriendIDs,screenName))
}

# append rows to dataframe
append_to_df<-function(dt, elems)
{
  return(rbindlist(list(dt,  elems),use.names = TRUE))
}

Extract Friends and Followers for a User

In [ ]:
# Begin with a certain username
coreUserName <- "sociolegaltech"
twitterUser <- getUser(coreUserName)


# Extract Followers for the core user
twitterUser_follower_IDs <- twitterUser$getFollowers(retryOnRateLimit=10)
twitterUser_followers_df = rbindlist(lapply(
  twitterUser_follower_IDs,as.data.frame
))

# Extract Followers for the core user
twitterUser_Friend_IDs <- twitterUser$getFriends(retryOnRateLimit=10)
twitterUser_Friends_df = rbindlist(lapply(
  twitterUser_Friend_IDs,as.data.frame
))

Optional to Filter Dummy Accounts

In [ ]:
# filter dummy accounts
filtered_df <- subset(twitterUser_followers_df,
                      followersCount < 100 &
                        statusesCount > 100 &
                        statusesCount < 5000 &
                        protected==FALSE)

filtered_follower_IDs <- twitterUser_followers_df$screenName

filtered_Friends_IDs <- twitterUser_Friends_df$screenName

In [3]:
# prepare edge data frame for Follwers
edge_df<-data.frame(from=filtered_follower_IDs,
                    to=rep('sociolegaltech',
                           length(filtered_follower_IDs)),
                    stringsAsFactors=FALSE)

SyntaxError: invalid syntax (<ipython-input-3-0cf615c71237>, line 2)

Compose list of followers

In [ ]:
# Iterate and extract list of followers of followers
counter = 1
for(follower in filtered_follower_IDs){
  # fetch follower list for current user
  followerScreenNameList <- get_follower_list(follower)

  print(paste("Processing completed for:",
              follower,
              "(",counter,"/",
              length(filtered_follower_IDs),")"
  ))
  # append to edge list
  edge_df <- append_to_df(edge_df,list(from=followerScreenNameList,
                                       to=rep(follower,
                                              length(followerScreenNameList))))
  counter <- counter + 1
}

In [ ]:
# prepare edge data frame for Friends
edge_df<-data.frame(from=filtered_Friends_IDs,
                    to=rep('sociolegaltech',
                           length(filtered_Friends_IDs)),
                    stringsAsFactors=FALSE)